# Using PySINDy to identify 2nd order differential systems

In the first underlying cell, the necessary imports and definitions are made.

In [3]:
import pysindy as ps
import numpy as np
import matplotlib.pyplot as plt
from pysindy.feature_library import ConcatLibrary, CustomLibrary

x = np.load('earth_pos.npy')
x = np.delete(x, 2, 1) # deletes z-coordinates
v = np.load('earth_vel.npy')
v = np.delete(v, 2, 1) # deletes z-coordinates

t = np.arange(0, x.shape[0], 1) # timesteps in days

functions = [lambda x,y : x/(x**2+y**2)**(3/2), lambda x,y : y/(x**2+y**2)**(3/2)] # The specific functions we're looking for
lib_custom = CustomLibrary(library_functions=functions) # defines the custom library we want to use
lib_concat = ConcatLibrary([lib_custom])

The following cell includes the model optimization and model identification. 

In [4]:
optimizer = ps.STLSQ(threshold=1e-15, fit_intercept=True)

model = ps.SINDy( 
        feature_library = lib_concat, 
        optimizer=optimizer,
        feature_names = ['x', 'y'])

xd = np.gradient(x[:, 0], t)
yd = np.gradient(x[:, 1], t)
dot = np.array([xd, yd]).T # .T is to transpose the array

print('Model where we define X_dot (which gives vel)')
model.fit(x, t=t, x_dot=dot)
model.print()

print('Model where we feed v to SINDy (which gives acc)')
model.fit(v, t=t)
model.print()

Model where we define X_dot (which gives vel)
x' = 0.017 f1(x,y)
y' = -0.017 f0(x,y)
Model where we feed v to SINDy (which gives acc)
x' = -0.001 f0(x,y) + 4.122 f1(x,y) + -0.001
y' = -4.112 f0(x,y) + 0.001 f1(x,y) + 0.015
